 پرهام عسکرزاده
 شماره دانشجویی : ۹۸۱۷۰۹۳۵

در این تمرین ما متن های مورد نظر بورسی را از سایت رهاورد ۳۶۵ جمع میکنیم و بر روی مراحل پیش پردازش متن را اعمال میکنیم و در آخر گزارشی از اطلاعات منن و همچنین تشخیص میدهیم که متن مورد نظر راجع کدام سهام بورسی به اطلاعات میدهد

 در این بخش پکیج های مورد نظر برای کرول کردن اطلاعات سایت رهاورد ۳۶۵ را اضافه میکنیم

In [8]:
from requests import ReadTimeout, TooManyRedirects, Timeout
from requests.exceptions import ChunkedEncodingError, ConnectionError
import json
import time
import requests
import random

 متن های مورد نظر ما از بخش گفتگو های سایت رهاورد ۳۶۵ کرول و جمع آوری میشوند
 قطعه کد زیر مربوط به کرولر تمرین است:

In [9]:
class TweetCrawler:

    def crawl_tweets(self, before_id=999999):
        url = f'https://rahavard365.com/api/moreposts?before_id={before_id}'
        headers = {
            'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
            'sec-fetch-site': 'same-origin',
            'sec-fetch-mode': 'cors',
            'sec-fetch-dest': 'empty'
        }
        while True:
            try:
                response = requests.get(url, headers=headers, timeout=25)
                if response.status_code == 429:
                    time.sleep(1)
                    continue
            except (Timeout, ReadTimeout, ConnectionError, TooManyRedirects, ChunkedEncodingError) as e:
                time.sleep(30)
                continue
            except Exception as e:
                raise e
            result = json.loads(response.text)
            return result['data'].get('posts')[random.randint(0, 19)]['body']


 با اجرا کردن این متد متن مورد نظر جمع شده و به صورت رندوم یکی از آنها نمایش داده می شود

In [11]:
TweetCrawler().crawl_tweets()

'سلام دوستان من امروز ورود کردم نظزتون رو صادقانه بگید'

 حال به بخش نرمالایز کردن متن می رسیم در این بخش علاوه بر استفاده از پکیج هضم از متد های تعریف شده ای که کار نرمال کردن و یکسان کردن متن و همجنین حذف stopowrd ها و emoji ها استفاده میکنیم

In [12]:
from hazm import Normalizer
import re

/usr/lib/python3.10/site-packages/nltk/parse/malt.py:183: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if ret is not 0:
/usr/lib/python3.10/site-packages/nltk/ccg/chart.py:274: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(children) is 2:


In [22]:
class SelfNormalizer:
    def __init__(self):
        self.normalizer = Normalizer()

    def remove_shapes_and_convert_emojis_to_unicode(self, text: str):
        shape_list = re.findall(r'[^\w\s,]', text)  # find all shape and emojis
        for shape in shape_list:
            shape_code = shape.encode('unicode-escape').decode('ASCII')
            if 'U000' in shape_code:  # if the shape is an emoji
                text = text.replace(shape, ' {} '.format(shape_code))
            else:
                text = text.replace(shape, ' ')
        return text

    def character_replacer(self, text: str):
        # Numbers
        text = re.sub(r'[٠⓪⓿０𝟶🄌]', '۰', text)
        text = re.sub(r'[١⓵❶➀➊꘡]', '۱', text)
        text = re.sub(r'[٢②２𝟐]', '۲', text)
        text = re.sub(r'[٣③３𝟛]', '۳', text)
        text = re.sub(r'[٤۴⓸➍𝟒𝟜]', '۴', text)
        text = re.sub(r'[٥⓹❺５𝟝]', '۵', text)
        text = re.sub(r'[٦۶⑥❻６𝟞𝟨]', '۶', text)
        text = re.sub(r'[٧➆➐７𝟟]', '۷', text)
        text = re.sub(r'[٨⑧❽８𝟖]', '۸', text)
        text = re.sub(r'[٩⑨❾𝟗]', '۹', text)

        # Alphabet
        text = re.sub(r'[آأ𞸀]', 'ا', text)
        text = re.sub(r'[بﭒﭓﭔﭕ𞸁]', 'ب', text)
        text = re.sub(r'[ﭗﭘﭙ]', 'پ', text)
        text = re.sub(r'[تﺖﭧ𞸕]', 'ت', text)
        text = re.sub(r'[ثﺙﺚ𞸶𞸖]', 'ث', text)
        text = re.sub(r'[ﺝﺞﺠ𞸢𞸂]', 'ج', text)
        text = re.sub(r'[چﭻﭼﮀ]', 'چ', text)
        text = re.sub(r'[حﺢﺣ𞸇]', 'ح', text)
        text = re.sub(r'[ﺦﺨ𞸗]', 'خ', text)

        text = re.sub(r'[دﺩﺪ]', 'د', text)
        text = re.sub(r'[ذﺫﺬ𞸘]', 'ذ', text)

        text = re.sub(r'[رﺭﺮ𞸓]', 'ر', text)
        text = re.sub(r'[زࢲﺯﺰ𞸆]', 'ز', text)
        text = re.sub(r'[ژﮊﮋ]', 'ژ', text)
        text = re.sub(r'[ﺱﺳﺴ𞸎𞸮]', 'س', text)
        text = re.sub(r'[ﺵﺶﺸ𞸴𞸔]', 'ش', text)
        text = re.sub(r'[ص𞸱𞸑]', 'ص', text)
        text = re.sub(r'[ضﻀ𞸹𞸙]', 'ض', text)
        text = re.sub(r'[ﻂﻃ𞸈]', 'ط', text)
        text = re.sub(r'[ﻆ𞸚]', 'ظ', text)
        text = re.sub(r'[عﻉﻊﻌ𞸯𞸏]', 'ع', text)
        text = re.sub(r'[ﻎﻏﻐ𞸻𞸛]', 'غ', text)
        text = re.sub(r'[ف𞸞𞸐]', 'ف', text)
        text = re.sub(r'[ﻖﻘ𞸟𞸒]', 'ق', text)
        text = re.sub(r'[گﮓﮔﮕ]', 'گ', text)
        text = re.sub(r'[كﮑ𞸊𞸪]', 'ک', text)
        text = re.sub(r'[ﻝﻞﻟ𞸋]', 'ل', text)
        text = re.sub(r'[مﻡﻤ𞸬𞸌]', 'م', text)
        text = re.sub(r'[ﻥ𞸍𞸭]', 'ن', text)
        text = re.sub(r'[ﻭﻮ𞸅ۅﯠ]', 'و', text)
        text = text.replace('وو', 'و')
        text = re.sub(r'[هﮪﻪ𞸤ﻫﻬ]', 'ه', text)
        text = re.sub(r'[ةﺔ]', 'ه', text)
        text = re.sub(r'[ىﻯﻰ𞸉ﯨﯩ]', 'ی', text)
        text = text.replace('ئی', 'یی')
        text = re.sub(r'[ئﺉﺋ]', 'ئ', text)
        text = re.sub(r'[ءﺀ۽]', 'ء', text)
        text = text.replace('﷼', ' ریال ')

        text = text.replace(' می ', ' می\u200c')
        text = text.replace(' نمی ', ' نمی\u200c')
        text = text.replace(' برمی ', ' برمی\u200c')
        text = text.replace(' برنمی ', ' برنمی\u200c')

        # Whitespace
        text = re.sub(r'(\r|\f|\v|\\r|\\n)+', '\n', text)
        text = re.sub(r'\s?\n\s+', '\n', text)
        text = re.sub(r'[\t]+', ' ', text)
        text = re.sub(r' {2,}', ' ', text)

        # semi-space
        text = text.replace('&zwnj;', '\u200c')
        text = re.sub(r'[\u2000-\u200f]+', "\u200c", text)
        return text

    def remove_stopword(self, text: str):
        stop_words = ['از', 'به ', 'با', ' نه ', 'را', 'که ', 'بود', 'است', 'هست', 'شد', ' در ', 'اگر ', 'همچنین ',
                      'چنین ', 'داشت']
        for word in stop_words:
            text = re.sub(word, ' ', text)
        return text

    def normalize_with_hazm(self, text: str):
        return self.normalizer.normalize(text)

    def complete_normalize(self, text: str) -> str:
        text = self.remove_shapes_and_convert_emojis_to_unicode(text)
        text = self.character_replacer(text)
        text = self.normalizer.normalize(text)
        text = self.remove_stopword(text)

        return text.strip()

 در این بخش اموجی ها و اشکال که کاربردی برای ما ندارند از متن مورد نظر حذف می شوند

In [14]:
SelfNormalizer().remove_shapes_and_convert_emojis_to_unicode()

'سلام دوستان من امروز ورود کردم نظزتون رو صادقانه بگید'

 این متد همان کار نرمال کردن را انجام میدهد که بدون استفاده از هرگونه پکیجی نوشته شده

In [21]:
SelfNormalizer().character_replacer()

'سلام دوستان من امروز ورود کردم نظزتون رو صادقانه بگید'

 در این متد stopword های مورد نظر از متن حذف میشوند

In [16]:
SelfNormalizer().remove_stopword()

'سلام دوستان من امروز ورود کردم نظزتون رو صادقا بگید'

 در این بخش با استفاده از پکیج هضم متن را نرمالایز میکنیم

In [23]:
SelfNormalizer().normalize_with_hazm()

'سلام دوستان من امروز ورود کردم نظزتون رو صادقانه بگید'

 این متد تجمیعی از متد های بالاست که متن را میگیرد و عمل های فوق را روی متن مورد نظر انجام میده

In [24]:
SelfNormalizer().complete_normalize()

'سلام دوستان من امروز ورود کردم نظزتون رو صادقانه بگید'

 بعد از نرمال کردن متن نوبت به گرفتن اطلاعات از متن نرمال شده میباشد
 ما در این کلاس اطلاعاتی قبیل : منشن ها و تعداد آنها ، هشتگ ها و تعداد آنها ، عدد ها و تعداد آنها داخل متن و همچنین نقش کلمات در جمله و همچنین توکن های جمله را خروجی میدهیم

In [44]:
from collections import Counter
from hazm import POSTagger, SentenceTokenizer

In [45]:
class Analyzer:
    def __init__(self):
        self.sign = {",", ".", "،", "?", "؟", "!", "!", "#", "*", "(", ")", "[", "]", "{", "}", " "}
        self.sentence_tokenizer = SentenceTokenizer()
        self.pos_tagger = POSTagger()

    def mentions(self, text: str):
        return list(set(re.findall('@([\w_.]+)', text)))

    def hashtags(self, text: str):
        return list(set(re.findall('#([\w_]+)', text)))

    def numbers(self, text: str):
        number = ''
        numbers = []
        for letter in text:
            try:
                int(letter)
            except:
                if len(number) != 0:
                    numbers.append(number)
                    number = ''
                continue
            number += letter
        return numbers

    def letter_info(self, text: str):
        letters = []
        namads = []
        for letter in text:
            if letter not in self.sign:
                letters.append(letter)
            elif letter != " ":
                namads.append(letter)
        return {"letter_count": len(letters),
                "sign_count": len(namads)}

    def info_of_text(self, text: str):
        return {
                'numbers_info': {
                    'numbers': self.numbers(text),
                    'count': len(self.numbers(text))},
                'letters_count': self.letter_info(text),
                'hashtags_info': {
                    'items': self.hashtags(text),
                    'count': len(self.hashtags(text))},
                'mentions_info': {
                    'items': self.mentions(text),
                    'count': len(self.mentions(text))}
                }

    def tokenizer(self, text: str):
        return self.sentence_tokenizer.tokenize(text)

    def postagger(self, text):
        return self.pos_tagger.tag(text)

    def rate_keywords(self, text):
        rate = Counter(text.split())
        all = sum(dict(rate).values())
        info_rate = {}
        for word in rate:
            info_rate[word] = round(float((rate[word] / all) * 100), 2)
        return info_rate

 در این بخش ما منشن های داخل متن و همچنین تعداد آنها رو خروجی میگیریم

In [46]:
Analyzer().mentions('سلام دوستان من امروز ورود کردم نظزتون رو صادقانه بگید')

Exception ignored in: <function Model.__del__ at 0x7f4acadeda20>
Traceback (most recent call last):
  File "/usr/lib/python3.10/site-packages/wapiti/api.py", line 289, in __del__
    if _wapiti and self._model:
AttributeError: 'Model' object has no attribute '_model'
Exception ignored in: <function Model.__del__ at 0x7f4acadeda20>
Traceback (most recent call last):
  File "/usr/lib/python3.10/site-packages/wapiti/api.py", line 289, in __del__
    if _wapiti and self._model:
AttributeError: 'Model' object has no attribute '_model'


ArgumentError: argument 2: <class 'TypeError'>: wrong type

 در این بخش ما هشتگ های داخل متن و همچنین تعداد آنها رو خروجی میگیریم

In [30]:
Analyzer().hashtags()

ArgumentError: argument 2: <class 'TypeError'>: wrong type

 در این بخش ما عدد های داخل متن و همچنین تعداد آنها رو خروجی میگیریم

In [ ]:
Analyzer().numbers()

 در این بخش ما تعداد حروفو نشانه های داخل متن را خروجی میگیریم

In [ ]:
Analyzer().letter_info()

 این بخش اطلاعات متد های بالا رابه صورت تجمیعی و یکجا به کاربر نمایش می دهد

In [36]:
Analyzer().info_of_text()

ArgumentError: argument 2: <class 'TypeError'>: wrong type

 دراین بخش با استفاده از پکیج هضم جمله را به توکن ها تقسیم میکنیم

In [ ]:
Analyzer().tokenizer()

 در این بخش نقش دستوری کلمات در متن با استفاده از پکیج هضم مشخص میشود

In [ ]:
Analyzer().postagger()

 در این متد ما درصد کلمات به کار رفته در متن را تشخیص می دهیم که این به ما کمک میکند که بتوانیم راجع سهم موردنظر در آینده اطلاعات دقیق تری از قبیل تشخیص میزان رشد ، نزول ، نوسان سهام بگیریم

In [42]:
Analyzer().rate_keywords('سلام دوستان من امروز ورود کردم نظزتون رو صادقانه بگید')

ArgumentError: argument 2: <class 'TypeError'>: wrong type

 در آخر با استفاده از قواعد زبان منظم نام سهم ها و نماد ها را داخل متن تشخیص میدهیم

In [40]:
def select_symbol_of_bourse(content: str):
        symbols = {'وتجارت', 'وپارس', 'فملی', 'وبصادر', 'کترام', 'فاذر', 'شراز', 'چکاپا', 'آپ', 'لابسا', 'کاما',
                   'پکویر', 'ثبهساز', 'کلر', 'پترول',
                   'ورنا', 'لکما', 'بترانس', 'کسرا', 'خفنر', 'ذوب', 'خدیزل', 'شستا', 'شاراک', 'فولاد', 'شپترو',
                   'وشهر',
                   'قاسم', 'پالایش', 'خساپا', 'پاسا', 'فسبزوار', 'وبرق', 'غزر', 'سفار', 'نوری', 'زگلدشت', 'ولساپا',
                   'وغدیر', 'سپید', 'وآیند', 'بکاب', 'وسالت', 'کیسون', 'تپکو', 'بجهرم', 'فروی', 'ختراک', 'همراه',
                   'غبشهر',
                   'غنوش', 'کیمیاتک', 'فلوله', 'تفارس-پذیره', 'آرام', 'خفولا', 'بالاس', 'غدشت', 'ثشاهد',
                   'کتوکا', 'کفپارس', 'زماهان', 'شفن', 'دی', 'خپارس', 'غصینو', 'مادیرا', 'زاگرس', 'قچار', 'کرمان',
                   'شکلر', 'شپلی', 'خکرمان', 'کدما', 'طلا', 'خنصیر', 'وهامونح', 'شلرد', 'برکت', 'کمند', 'وسین',
                   'سجام',
                   'مفاخر', 'شوینده', 'خکار', 'شیشه01ن', 'افق', 'شپدیس', 'خاور', 'تمحرکه', 'کالا', 'صبا', 'سیمرغ',
                   'سمگا',
                   'زگلدشتح', 'خکمک', 'فزرین', 'فنفت', 'رتاپ', 'دارا یکم', 'خگستر', 'وآذر', 'ساذری', 'خودکفا',
                   'غالبر',
                   'بزاگرس', 'غشهداب', 'وساپا', 'قنیشا', 'کگاز', 'فولای', 'وپست', 'خودرو', 'شگویا', 'خلنت', 'ثاخت',
                   'شپنا', 'شتران', 'غگرجی', 'وبملت', 'سیتا', 'گشان', 'وگردش', 'وسدید'}

        symbol_of_content = []
        for symbol in symbols:
            if symbol == 'دی':
                if content.startswith('دی ') or content.endswith(' دی') or ' دی ' in content:
                    symbol_of_content.append(symbol)
                else:
                    continue
            if symbol in content:
                symbol_of_content.append(symbol)

        if len(symbol_of_content):
            symbol_of_content = 'nothing'
        return symbol_of_content

 با وارد کردن متن مورد نظر در تابع زیر سهم های تشخیص داده شده در متن نمایش داده میشود

In [41]:
select_symbol_of_bourse()

[]

 در این تمرین سعی شده خواسته های مورد نظر در داک تمرین به خوبی پیاده سازی شود
 با تشکر از حسن نظر شمااستاد گرامی